# Introduction
We analyse political data by examining the voting patterns of the 111th US House of Representatives during the years 2009-2011. We use the *rollcall-votes.Rdata* file, which records whether each of the 445 House members voted "yea" = 1, "nay" = -1, or abstained = 0 on 1,647 different legislative decisions. In addition, the file *rollcall-members.Rdata* enriches our dataset by providing additional details about the members, including their party affiliation (Republican or Democrat) and the US state they represent.

This tutorial promises to give you not only the technical skills to manipulate and analyse large datasets, but also insights into political dynamics and voting behaviour. By the end of this tutorial, you will have hands-on experience in data preparation, exploration and advanced analysis using unsupervised ML. nance.

## Load Data

This section of the script is dedicated to loading the essential datasets for our analysis. It retrieves the `rollcall-votes.Rdata` and `rollcall-members.Rdata` files from specified directories, effectively importing the voting records and member details of the 111th US House of Representatives. Upon successful loading, a confirmation message, "Data loaded," is printed to signal that the datasets are ready for use.

In [ ]:
base_path <- "~/Github_/M02-Machine-Learning"
base_path <- path.expand(base_path)
setwd(base_path)


In [ ]:
########################  Load Data  ########################

### Load data
load("self_study_tutorial/scripts_and_data/rollcall-votes.Rdata")
load("self_study_tutorial/scripts_and_data/rollcall-members.Rdata")
print('Data loaded.')

##############################################################

In [ ]:
head(members)

In [ ]:
head(votes)

# Data Exploration

This section focuses on analyzing party composition and voting behaviors in the 111th US House of Representatives. We start by displaying counts and proportions of Democrats, Republicans, and one notable politician. Next, we examine voting participation by counting missing votes and visualizing this data with a histogram to analyze absenteeism patterns. Additionally, we count 'yea' and 'nay' votes, plotting these against each other by party to highlight differences in voting behavior across party lines. This exploration helps us understand the dataset's structure and the political dynamics at play.

**a) How many Democrats and Republicans are in the House? Who has the majority? Report the special case Democrat-Republican separately.**

In [ ]:
print('# Counts of Democrats, Republicans and one special politician')
table(members$party)

print('# Shares of Democrats, Republicans and one special politician')
round(table(members$party)/nrow(members),3)

In [ ]:
members[members$party == "DR",]

**b) Generate a variable for the number of votes each politician missed. How many politicians voted in all votings? Plot a histogram of the shares of missed votings with 100 bins.**

In [ ]:
# Count missing votings for each politician and plot the counts
missings <- rowSums(votes==0)

# No. politicians who always voted
sum(missings == 0)

In [ ]:
# Shares of missing votings
s_missings <- missings/(ncol(votes))

# Histogram with 100 bins
hist(s_missings, breaks = 100)

**c) Generate variables for the number of times each politician voted “yea” and “nea”. Make a scatter plot of the number of “nea” and “yea”. Use diﬀerent colors to diﬀerentiate the points in the scatter plot by the party aﬃliation. Can we claim based on these results that the party could be a good predictor for the voting behavior? What information do we lose by aggregation of the votes?**

In [ ]:
# Counts - yes and nos
yeas <- rowSums(votes[,(1:ncol(votes))]== 1)
nays <- rowSums(votes[,(1:ncol(votes))]== -1)

# Plots - Party
plot(yeas, nays, col = members$party)
legend('topleft', legend = levels(members$party), col = 1:3,  pch = 1)

# Principal Component Analysis (PCA)

**a) Run a principal component analysis on the votes data set. How many principal components are there?**

In [ ]:
# PCA
pr.out = prcomp(votes , center = TRUE, scale = TRUE)

# No of principal components
dim(pr.out$rotation)[2]

**b) Calculate the proportion of variance explained by each principal component and plot the proportions for the ﬁrst 10 principal components.**

In [ ]:
# variance explained by each component
pr.var = pr.out$sdev^2

# Proportion of variance explained
pve=pr.var/sum(pr.var)

# Print first 10 PC
pve[1:10]

In [ ]:
# Plot the first 10 PC
barplot(pve[1:10], xlab=" Principal Component ", ylab=" Proportion of Variance Explained ", ylim=c(0,1))
barplot(cumsum(pve[1:10]), xlab=" Principal Component ", ylab ="Cumulative Proportion of Variance Explained ", ylim=c(0,1))

**c) Plot the ﬁrst two principal components and use color to diﬀerentiate the observations by the party aﬃliation.**

In [ ]:
# Plot the first two principal components, color the party membership
plot(pr.out$x[,1], pr.out$x[,2], xlab = "PC1", ylab = "PC2", col = members$party, main = "Top two PC directions")
legend('bottomright', legend = levels(members$party), col = 1:3,  pch = 1)

**d) With the help of the ﬁrst principal component ﬁnd politicians on the far right (very conservative) and far left (very liberal).**

In [ ]:
## Far right (very conservative)
head(sort(pr.out$x[,1]))

In [ ]:
## Far left (very liberal)
head(sort(pr.out$x[,1], decreasing=TRUE))

**e) Find the votings which have the most extreme loadings for the second principal component. Analyze the voting behavior in these votings and come up with the interpretation of the second component.**

In [ ]:
# PC 2
head(sort(pr.out$x[,2]))
# No clear pattern based on party and state information

In [ ]:
# Look at the largest loadings in PC2 to discern an interpretation.
loadings <- pr.out$rotation
loadings[order(abs(loadings[,2]), decreasing=TRUE)[1:5],2]

In [ ]:
# Analyze voting behavior
table(votes[,1146])
table(votes[,658])
table(votes[,1090])

In [ ]:
# Either everyone voted "yea" or missed the voting.
# These votes all correspond to near-unanimous symbolic action.

# Mystery Solved: the second PC is just attendance!
head(sort(rowSums(votes==0), decreasing=TRUE))

<span style="color:red; font-weight:bold; font-size:xx-large;">
# Time for a breakout session 
</span>


# Clustering

**a) Run a k-means clustering procedure to detect 2 clusters in the data. How many Democrats and Republicans are in each cluster?**

In [ ]:
set.seed(11122019)

# K-means clustering with 2 clusters
km.out = kmeans(votes, 2, nstart = 20)
#km.out$cluster
# Tabulate party vs cluster
table(members$party, km.out$cluster)


In [ ]:
km.out$cluster

**b) Run a k-means clustering procedure for k ∈{2,...,20} and plot the within cluster sum of squared errors. Decide based on the plot what the optimal number of clusters is.**

In [ ]:
# How to analyze the optimal number of clusters

sse <- c()
sse[1] <- Inf

for (ind_cl in c(2:20)) {
  set.seed(3)
  km.out = kmeans (votes, ind_cl, nstart = 20)
  sse[ind_cl] = km.out$tot.withinss
}

plot(sse)
# Optimum 4-5 clusters

**c) Take the graph with two principal components and use color to visualize the clusters you found in the previous task.**

In [ ]:
# Plot the 5 clusters on the PC components graph
set.seed(3)
km.out = kmeans (votes, 5, nstart = 20)

# Plot the first two principal components color the party membership
plot(pr.out$x[,1], pr.out$x[,2], xlab = "PC1", ylab = "PC2", col = km.out$cluster, main = "Top two PC directions with 5 clusters")
legend('bottomright', legend = c("Cluster 1", "Cluster 2", "Cluster 3", "Cluster 4", "Cluster 5"), col = 1:5,  pch = 1)

**d) Run a k-means clustering procedure to detect 6 clusters with nstart = 1 and nstart = 20. Print out the within cluster sum of squared errors for both nstart values. What can you say about the importance of nstart on the convergence to the global optimum?**

In [ ]:
# Analyzing how the number of starts work
set.seed (3)
print('With nstart = 1')
km.out = kmeans (votes,6, nstart = 1)
km.out$tot.withinss

print('With nstart = 20')
km.out =kmeans (votes,6, nstart = 20)
km.out$tot.withinss

print('With nstart = 100')
km.out =kmeans (votes,6, nstart = 100)
km.out$tot.withinss

print('With nstart = 500')
km.out =kmeans (votes,6, nstart = 500)
km.out$tot.withinss


<span style="color:red; font-weight:bold; font-size:xx-large;">
# Time for a breakout session 
</span>
